In [200]:
# import the required libraries
import docx2txt
import os
import re
import csv
import pandas as pd
import numpy as np
import act1

In [201]:
# Set variables for the project (i.e. the input location of the file to be processed and the output location) )

folderlink = '..//data//'
input_folder = 'input//'
input_file = os.path.join(folderlink+input_folder, 'Repertoriummetoudepersoonsnummers.docx')
folder_output = 'output//'
output_txt = folderlink+folder_output+'output.txt'
output_csv = folderlink+folder_output+'output_file.csv'

In [202]:
# Panda settings for showing data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [203]:
# Converting the docx to a text file and remove all unecessary rows.

# Use docx2txt library to extract text from .docx file
text = docx2txt.process(input_file)

# Remove excessive whitespaces
data = ' '.join(text.split())

Manual updates in the dataset:

1.  FROM: "N.N. "de oude vicarius">pred. Lichtenvoorde 1602 tot 1615.<20871>"
    TO: "N.N. "de oude vicarius" pred. Lichtenvoorde 1602 tot 1615.<20871>" 
    REASON: since it contained the > character which is used to define ID fields. 
 
2.  FROM: "Bosch; Cornelius Geb. Utrecht 1634; pred. Renswoude 16 dec. 1656, Maasland 15 april 1663, Brielle 30 jan. 1667, Alkmaar 1667, 's Gravenhage 5 juli 1676, emer. 1713,;overl. 28 maart 1715.<1185>"
    TO:Bosch; Cornelius
Geb. Utrecht 1634; pred. Renswoude 16 dec. 1656, Maasland 15 april 1663, Brielle 30 jan. 1667, Alkmaar 1667, 's Gravenhage 5 juli 1676, emer. 1713, overl. 28 maart 1715.<1185>
    REASON: Since it splitted the string based on ",;overl"

3.  FROM: Leeuwen, van, Cornelis [z.v. Cornelis]
Geb. Hazerswoude 1611; pred. Boskoop en Middelburg 1637, overl. 1681.<5778>
    TO: Leeuwen, van; Cornelis [z.v. Cornelis]
Geb. Hazerswoude 1611; pred. Boskoop en Middelburg 1637, overl. 1681.<5778>
    REASON: there was no ; between the name and surname (including infix)

4.  FROM: Peenen, van, Marcus
Gedoopt Leiden 31 aug. 1642; pred. Koudekerk aan den Rijn 2 sept. 1668, Leiden 1680, begraven 1 febr. 1696.<7388>
    TO: Peenen, van; Marcus
Gedoopt Leiden 31 aug. 1642; pred. Koudekerk aan den Rijn 2 sept. 1668, Leiden 1680, begraven 1 febr. 1696.<7388>
    REASON: there was no ; between the name and surname (including infix)

5.  FROM: Knuyt, de (Kuntius), Elias
Geb. Middelburg yyy; pred. Oude Niedorp en Veenhuizen (NH) mei 1628, Sint Annaland 1630, Westkapelle 7 maart 1641, overl. --of vertrokken?-- 1642.<5381>
    TO: Knuyt, de (Kuntius); Elias
Geb. Middelburg yyy; pred. Oude Niedorp en Veenhuizen (NH) mei 1628, Sint Annaland 1630, Westkapelle 7 maart 1641, overl. --of vertrokken?-- 1642.<5381>
    REASON: there was no ; between the name and surname (including infix)

6.  FROM: Leonardis, de, Paulus
Geb. Keulen yyy; pred. Bacharach (Pfalz) 16.., Kampen 1620, overl. 1649.<5836>
    TO: Leonardis, de; Paulus
Geb. Keulen yyy; pred. Bacharach (Pfalz) 16.., Kampen 1620, overl. 1649.<5836>
    REASON: there was no ; between the name and surname (including infix)

7.  FROM: Tamerus; Henricus
Geb. xxx ca. 1540; voorheen Lutheraan;--onwettig pred. Heteren en Randwijk ca. 1600-1602; pred. Eethen en Meeuwen 1606, Doeveren, Gansoyen en Genderen 1610, afgezet als remonstrant 1619; overl. Heusden.<21206>
    TO: Tamerus; Henricus
Geb. xxx ca. 1540; voorheen Lutheraan; onwettig pred. Heteren en Randwijk ca. 1600-1602; pred. Eethen en Meeuwen 1606, Doeveren, Gansoyen en Genderen 1610, afgezet als remonstrant 1619; overl. Heusden.<21206>
    REASON: -- conflicted created a line break. 




In [204]:
data = data.replace('overl. 28 maart 1715.<1185>', ' overl. 28 maart 1715.<1185>').replace('>pred.', ' pred.').replace('Leeuwen, van, Cornelis [z.v. Cornelis]','Leeuwen, van; Cornelis [z.v. Cornelis]',).replace('Peenen, van, Marcus','Peenen, van; Marcus').replace('Knuyt, de (Kuntius), Elias','Knuyt, de (Kuntius); Elias').replace('Leonardis, de, Paulus','Leonardis, de; Paulus').replace('Lutheraan;--onwettig','Lutheraan; onwettig')

In [205]:
# Replace semicolons with newlines and add semicolons around < and > since these identify the IDs
data = data.replace(';', ';\n').replace(';\n ', '; ').replace('>', '>;\n ').replace('<', ';<')
lines = data.split('\n')

lines = [line for line in lines if not line.startswith('; ;') and not line.startswith('; ')]
data = '\n'.join(lines)
lines = data.strip().split('\n')
data = '\n'.join([line.lstrip() for line in lines])


In [206]:
lines = data.split('\n')

with open(output_txt, "w", encoding='utf-8') as file:
    for line in lines:
        result_string = act1.replace_pred_count(line)
        file.write(result_string + "\n")

In [207]:
columns = ("Geb.", 
 "0 pred.",
 "1 pred.",
 "2 pred.",
 "3 pred.",
 "4 pred.",
 "overl.", 
 "Gedoopt", 
 "legerpred.",
 "pastoor",
 "garnizoenspred.",
 "emer.",
 "begraven",
 "conrector",
 " rector",
 "monnik",           
 "schoolmeester",
 "hoogleraar",
 "chirurgijn",
 "praeceptor",
 "ziekentrooster",
 "vlootpred.",
 "legerpred.",
 "ambassadepred."           
)          
    

In [208]:
for column in columns:
    with open(output_txt, 'r', encoding='utf-8') as f:
        lines = f.readlines()        
    with open(output_txt,'w', encoding='utf-8') as f:
        for line in lines:
            if "; "+column in line:
                f.write(line)
            elif column in line:
                line = line.replace(column, ";"+column)
                f.write(line)
            else:
                f.write(line)

In [209]:
# Define the headers for the output file
headers = ['surname_temp', 'first_name', 'Field1', 'Field2', 'Field3', 'Field4', 'Field5', 'Field6', 'Field7', 'Field8', 'Field9', 'Field10', 'Field11','Field12','Field13','Field14','Field15','Field16','Field17']

with open(output_txt, 'r', encoding='utf-8') as infile, open(output_csv, 'w', newline='', encoding='utf-8' ) as outfile:
    reader = csv.reader(infile, delimiter=';')
    writer = csv.writer(outfile, delimiter=';')
    
    # Write the headers to the output file
    writer.writerow(headers)

    # Loop through each row in the input file and write it to the output file with 12 fields
    for row in reader:
        # Create a new row with 12 fields by extending the current row with empty values
        new_row = row + [''] * (12 - len(row))
        writer.writerow(new_row)

In [210]:
df = pd.read_csv(output_csv, sep=';', encoding='utf-8')

In [211]:
#In the file all IDs are stored between < and > therefore 
for column in df.columns:
    # Check if any value in the column contains '<'
    if df[column].astype(str).str.contains('<').any():
        # Copy the values containing '<' to the 'test' column
        df.loc[df[column].astype(str).str.contains('<'), 'drc_id'] = df[column]


In [212]:
df['drc_id'] = df['drc_id'].str.replace('>', '').str.replace('<','')

In [213]:
for column in columns:
    df[column] = df.apply(lambda row: row[row.astype(str).str.contains(column)].iloc[0] if any(row.astype(str).str.contains(column)) else '', axis=1)

In [ ]:
df.head()

In [214]:
dft = df

In [215]:
df['original_input'] = df['surname_temp'].fillna('') + df['first_name'].fillna('') + df['Field1'].fillna('') + df['Field2'].fillna('')+ df['Field3'].fillna('')+ df['Field4'].fillna('')+ df['Field5'].fillna('')+ df['Field6'].fillna('')+ df['Field7'].fillna('')+ df['Field8'].fillna('')+ df['Field9'].fillna('')+ df['Field10'].fillna('')+ df['Field11'].fillna('')+ df['Field12'].fillna('')+ df['Field13'].fillna('')+ df['Field14'].fillna('')+ df['Field15'].fillna('')+ df['Field16'].fillna('')+ df['Field17'].fillna('')


In [216]:
df['original_input'] = df['original_input'].str.replace('0 pred. ', ' pred. ').str.replace('1  pred.',' pred. ').str.replace('2 pred.',' pred. ').str.replace('3  pred.',' pred. ').str.replace('4  pred.',' pred. ')

In [217]:
for column in columns:
    df[column] = df[column].str.replace(column, '')

In [ ]:
df.head()

In [218]:
df['name_info_family'] =df['first_name'] .str.extract(r'\[(.*?)\]')


In [219]:
df['nickname'] =df['surname_temp'] .str.extract(r'\((.*?)\)')


In [220]:
columns_rename = {
    'Geb.': 'birth', 
    #' pred.': 'minister', 
    'overl.': 'death', 
    'Gedoopt':'baptized', 
    'legerpred.':'legerpredikant',
    'pastoor':'pastoor',
    'garnizoenspred.':'garnizoenspredikant',
    "emer.":'emeritus_status',
    "begraven":'burried',
    "conrector":'conrector',
    " rector":'rector',
    "monnik":'monnik',           
    "schoolmeester":'schoolmeester',
    "hoogleraar":'hoogleraar',
    "chirurgijn":'chirurgijn',
    "praeceptor":'praeceptor',
    "ziekentrooster":'ziekentrooster',
    "vlootpred.":'vlootpredikant',
    "legerpred.":'legerpredikant',
    "ambassadepred.":'ambassadepredikant'}

In [221]:
# Rename the columns
df = df.rename(columns=columns_rename)
new_columns = list(columns_rename.values())


In [222]:
array_drop = [i for i in range(1, 18)]
for dropid in array_drop:
    column_dropid = 'Field'+str(dropid)
    df = df.drop(column_dropid, axis=1)

In [223]:
def extract_year(text):
    match = re.search(r'\d{4}', text)
    if match:
        return match.group(0)
    else:
        return None

In [224]:
df['minister'] = df['0 pred.']+ ','+df['1 pred.']+ ','+df['2 pred.']+ ','+df['3 pred.']+ ','+df['4 pred.']

In [225]:
df = df.drop(['0 pred.','1 pred.','2 pred.','3 pred.','4 pred.'], axis=1)

In [226]:
function_year = [word for word in new_columns if word != 'minister']

print(function_year)

['birth', 'death', 'baptized', 'legerpredikant', 'pastoor', 'garnizoenspredikant', 'emeritus_status', 'burried', 'conrector', 'rector', 'monnik', 'schoolmeester', 'hoogleraar', 'chirurgijn', 'praeceptor', 'ziekentrooster', 'vlootpredikant', 'ambassadepredikant']


In [227]:
df.head()

,surname_temp,first_name,drc_id,birth,death,baptized,legerpredikant,pastoor,garnizoenspredikant,emeritus_status,burried,conrector,rector,monnik,schoolmeester,hoogleraar,chirurgijn,praeceptor,ziekentrooster,vlootpredikant,ambassadepredikant,original_input,name_info_family,nickname,minister
0,"Aalburg, van",Johannes,1,Zierikzee ca. 1717,14 maart 1777.,,,,,,,,,,,,,,,,,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>",NaN,NaN,"Oudkarspel 30 juni 1743, ,,,,"
1,Aalst,Wilhelmus,4,,19 dec. 1700.,Biggekerke 5 jan. 1664,,,,,,,,,,,,,,,,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>",NaN,NaN,"Aardenburg 22 mei 1695, ,,,,"
2,"Aalst, van",Cornelius,2,Castricum ca. 1686,Amsterdam 27 aug. 1756.,,,,,1751,,,,,,,,,,,in Parijs maart tot dec. 1715,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>",NaN,NaN,"Kalslagen ber. 21 febr. 1717, ,,,,"
3,"Aalst, van",Gerardus,3,xxx sept. 1678,29 juni 1759.,,,,,1755,,,,,,,,,,,,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",NaN,NaN,"Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, ,,,,"
4,Aalstius,Henricus,5,'s-Hertogenbosch (?) yyy,Limmen 15 maart 1736.,,,,,sept. 1733,,,,,,,,,,,,"Aalstius Henricus Geb. 's-Hertogenbosch (?) yyy pred. Castricum en Heemskerk nov. 1700, emer. sept. 1733 overl. Limmen 15 maart 1736.<5>",NaN,NaN,"Castricum en Heemskerk nov. 1700, ,,,,"


In [228]:
for year in function_year:
    fld_year = 'year_' +year
    df[fld_year] = df[year].apply(lambda x: extract_year(x))    

In [229]:
for year_accu in function_year:
    accu_fld_year = 'accu_year_' +year_accu
    df[accu_fld_year] = ''
    # Iterate over each row in the DataFrame
    for index, row in df.iterrows():
        # Check if the string contains "ca." (case-insensitive)
        if 'ca.' in row[year_accu].lower():
            # If found, set the value of the "accuracy" column to "circa"
            df.at[index, accu_fld_year] = 'circa'
            
    

In [230]:
for remark in function_year:
    remarks_field = 'remarks_'+remark
    df[remarks_field] = df[remark]

In [ ]:
df.head()

In [231]:
months =(" januari ", 
 " februari ",
 " maart ",
 " april ",
 " mei ",
 " juni ",
 " juli ",
 " augustus ",
 " september ",
 " oktober ",
 " november ",           
 " december ",
 "jan. ",
 "feb. ",
 "mrt. ",
 "apr. ",
 "jun. ",
 "jul. ",
 "aug. ",
 "sept. ",
 "sep. ",
 "okt. ",
 "nov. ",
 "dec. ",
 "yyy",
 "xxx",
 "ca.",
 "febr."        
) 


In [232]:
for column_strip in function_year:   
    for month in months:
        df[column_strip] = df[column_strip].str.replace(month, '')


In [233]:
for column_strip in function_year:
    df[column_strip] = df[column_strip].apply(lambda x: re.sub(r'[\d\.]', '', x))

In [ ]:
df.head()

In [234]:
df['surname_temp']= df['surname_temp'].str.replace(r'\(.*\)', '', regex=True)
df['first_name']= df['first_name'].str.replace(r'\[.*\]', '', regex=True)

In [ ]:
df.head()

In [235]:
df[['surname', 'infix']] = df['surname_temp'].str.split(',', expand=True)

In [236]:
df['first_letter'] = df['first_name'].astype(str).apply(lambda x: x[1])

In [ ]:
df.head()

In [237]:
df = df.drop(['surname_temp'], axis=1)

In [239]:
# This join field is only created for the linkages with DM should be transfered lateron to there

df['join_name'] = df['surname']+df['first_name']+df['infix'].fillna('')
df['join_name'] = df['join_name'].str.replace("  "," ")

In [240]:
df.head()

,first_name,drc_id,birth,death,baptized,legerpredikant,pastoor,garnizoenspredikant,emeritus_status,burried,conrector,rector,monnik,schoolmeester,hoogleraar,chirurgijn,praeceptor,ziekentrooster,vlootpredikant,ambassadepredikant,original_input,name_info_family,nickname,minister,year_birth,year_death,year_baptized,year_legerpredikant,year_pastoor,year_garnizoenspredikant,year_emeritus_status,year_burried,year_conrector,year_rector,year_monnik,year_schoolmeester,year_hoogleraar,year_chirurgijn,year_praeceptor,year_ziekentrooster,year_vlootpredikant,year_ambassadepredikant,accu_year_birth,accu_year_death,accu_year_baptized,accu_year_legerpredikant,accu_year_pastoor,accu_year_garnizoenspredikant,accu_year_emeritus_status,accu_year_burried,accu_year_conrector,accu_year_rector,accu_year_monnik,accu_year_schoolmeester,accu_year_hoogleraar,accu_year_chirurgijn,accu_year_praeceptor,accu_year_ziekentrooster,accu_year_vlootpredikant,accu_year_ambassadepredikant,remarks_birth,remarks_death,remarks_baptized,remarks_legerpredikant,remarks_pastoor,remarks_garnizoenspredikant,remarks_emeritus_status,remarks_burried,remarks_conrector,remarks_rector,remarks_monnik,remarks_schoolmeester,remarks_hoogleraar,remarks_chirurgijn,remarks_praeceptor,remarks_ziekentrooster,remarks_vlootpredikant,remarks_ambassadepredikant,surname,infix,first_letter,join_name
0,Johannes,1,Zierikzee,,,,,,,,,,,,,,,,,,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>",NaN,NaN,"Oudkarspel 30 juni 1743, ,,,,",1717,1777,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,circa,,,,,,,,,,,,,,,,,,Zierikzee ca. 1717,14 maart 1777.,,,,,,,,,,,,,,,,,Aalburg,van,J,Aalburg Johannes van
1,Wilhelmus,4,,,Biggekerke,,,,,,,,,,,,,,,,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>",NaN,NaN,"Aardenburg 22 mei 1695, ,,,,",None,1700,1664,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,,,19 dec. 1700.,Biggekerke 5 jan. 1664,,,,,,,,,,,,,,,,Aalst,None,W,Aalst Wilhelmus
2,Cornelius,2,Castricum,Amsterdam,,,,,,,,,,,,,,,,in Parijstot,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>",NaN,NaN,"Kalslagen ber. 21 febr. 1717, ,,,,",1686,1756,None,None,None,None,1751,None,None,None,None,None,None,None,None,None,None,1715,circa,,,,,,,,,,,,,,,,,,Castricum ca. 1686,Amsterdam 27 aug. 1756.,,,,,1751,,,,,,,,,,,in Parijs maart tot dec. 1715,Aalst,van,C,Aalst Cornelius van
3,Gerardus,3,,,,,,,,,,,,,,,,,,,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",NaN,NaN,"Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, ,,,,",1678,1759,None,None,None,None,1755,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,,xxx sept. 1678,29 juni 1759.,,,,,1755,,,,,,,,,,,,Aalst,van,G,Aalst Gerardus van
4,Henricus,5,'s-Hertogenbosch (?),Limmen,,,,,,,,,,,,,,,,,"Aalstius Henricus Geb. 's-Hertogenbosch (?) yyy pred. Castricum en Heemskerk nov. 1700, emer. sept. 1733 overl. Limmen 15 maart 1736.<5>",NaN,NaN,"Castricum en Heemskerk nov. 1700, ,,,,",None,1736,None,None,None,None,1733,None,None,None,None,None,None,None,None,None,None,None,,,,,,,,,,,,,,,,,,,'s-Hertogenbosch (?) yyy,Limmen 15 maart 1736.,,,,,sept. 1733,,,,,,,,,,,,Aalstius,None,H,Aalstius Henricus


In [241]:
exclude_elements = ['birth', 'death', 'baptized', 'burried']

roles = [item for item in function_year if item not in exclude_elements]

In [242]:
for role in roles:
    accu_year = 'accu_year_'+role
    year = 'year_'+role
    role_remarks = 'remarks_'+role
    role_df = role
    role_df = df[['drc_id',role,year,accu_year,role_remarks]]
    columns_to_check = [role, year, accu_year, role_remarks]    
    role_df = role_df[role_df[columns_to_check].notna().all(axis=1)]
    new_column_names = {accu_year : 'accuracy_year', year : 'year', role_remarks : 'remarks'}
    role_df.rename(columns=new_column_names, inplace=True)
    role_df['role'] = role
    file_name = f'{folderlink+folder_output+role}_info.csv'  
    role_df.to_csv(file_name, sep=';', encoding='utf-8', index=False)

In [ ]:
column_list = df.columns.tolist()
print(column_list)

In [ ]:
drc_parent = df[['drc_id','first_name', 'infix', 'surname', 'first_letter', 'birth', 'death', 'baptized', 'burried', 'original_input', 'name_info_family', 'nickname', 'year_birth', 'year_death', 'year_baptized', 'year_burried', 'accu_year_birth', 'accu_year_death', 'accu_year_baptized', 'accu_year_burried', 'join_name']]
drc_parent.to_csv(folderlink+folder_output+'drc_data.csv', sep=';', encoding='utf-8', index=False)

In [ ]:
df.to_csv(folderlink+folder_output+'drc_data.csv', sep=';', encoding='utf-8', index=False)

From here we are going to make the parent child relationship with predikanten Ministers

Here the minister data is set as a child. 

In [ ]:
subset_pred = df[['drc_id', 'first_letter','minister','join_name','original_input']]

In [ ]:
subset_pred.head()

In [ ]:
df_expanded = subset_pred.assign(minister=subset_pred['minister'].str.split(','))

# Explode the 'pred.' column to create separate rows for each item
df_expanded = df_expanded.explode('minister')

In [ ]:
df_expanded.head()

In [ ]:
df_filtered = df_expanded[['drc_id','first_letter', 'minister','join_name','original_input']]

In [ ]:
childs = df_filtered[(df_filtered["minister"] !=" ") & (df_filtered["minister"] !="")]

In [ ]:
childs.head()

In [ ]:
childs['minister_year'] = childs['minister'].apply(lambda x: extract_year(x))

In [ ]:
childs['remarks_field'] = childs['remarks']
accu_fld_year = 'accu_year_' +'minster'
childs[accu_fld_year] = ''
# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    # Check if the string contains "ca." (case-insensitive)
    if 'ca.' in row[year_accu].lower():
        # If found, set the value of the "accuracy" column to "circa"
        df.at[index, accu_fld_year] = 'circa'

In [ ]:
childs.head()

In [ ]:
for month in months:
        childs['minister'] = childs['minister'].str.replace(month, '')

childs['minister'] = childs['minister'].apply(lambda x: re.sub(r'[\d\.]', '', x))

In [ ]:
childs.head()

In [ ]:
childs.to_csv(folderlink+folder_output+'minister_info.csv', sep=';', encoding='utf-8', index=False)

In [ ]:
os.remove(output_txt)
os.remove(output_csv)